In [9]:
import numpy as np
import pandas as pd
import operator
import pickle
import shelve
import re
from collections import Counter, defaultdict, OrderedDict, deque
import csv
import rdflib
from rdflib.namespace import OWL, RDF, RDFS
from rdflib import URIRef
import os

In [10]:
#####################################################
################ Initialization #####################
#####################################################

#ground truth metadata file
labeledFile = 'ebu3b_sensor_types_location.csv'
with open(labeledFile, 'rb') as fp:
    #Import list of points to a data frame
    #Pandas data frame allows you to store records and manipulate them: http://pandas.pydata.org/
    points_list = pd.DataFrame.from_csv(fp)
    points_list = points_list.set_index(keys='Unique Identifier')
    points_list.index.str.strip()

#Import lookup table that maps EBU3B descriptions to Brick tagsets
desc_tagset_map = pd.read_csv('ebu3b_description_to_tagset.csv').set_index('BACnet Description')

In [12]:
#####################################################
######## Initialize RDFlib graph and namespaces #####
#####################################################

#Namespaces
#Note: The URLs are fake
BRICK = rdflib.Namespace('http://buildsys.org/ontologies/brick#')
EBU3B = rdflib.Namespace('http://ucsd.edu/building/ontology/ebu3b#')
#RDF, RDFS and OWL have already been imported in the library initializations

#Initiate graph from base ttl file
g = rdflib.Graph()
g.bind('ebu3b', EBU3B)
g.bind('brick', BRICK)
g.parse('../../BuildingSchema/Brick.owl', format='turtle')

<Graph identifier=Nf126b24e65b446989882bdb2b9e73a8e (<class 'rdflib.graph.Graph'>)>

In [ ]:
#Note: This is not code. Writing it in code format to deal with Github formatting issues.
#I'm going to map the point with the following raw metadata. The "Actual Cooling Setpoint" maps to 
#"Zone Temperature Effective Cooling Setpoint" in the Brick tagset. This mapping is stored in 
#"ebu3b_description_to_tagset.csv" file. 
Unique Identifier: 505_0_3001627
BACnet Device ID                                    505
BACnet Object Type                                    0
BACnet Object ID                                3001627
Johnson Controls Name          EBU3B.RM-1100A..ACTCLGSP
BACnet Name                NAE 05 N2 2 VMA 103 ACTCLGSP
BACnet Description              Actual Cooling Setpoint
BACnet Unit                                          64
Ground Truth Point Type         Actual Cooling Setpoint
Point Ref                                           NaN
Location                                       RM-1100A
Equipment Type                                      VAV
Equipment Ref                                       103
Equipment Subsystems                                NaN
Building                                          EBU3B
Comments                                            NaN
BACnet Type Str                            Analog Input
Name: 505_0_3001627, dtype: object

In [13]:
#####################################################
######## Add triples and serialize them #############
#####################################################

#### Example for Point ####

###Adding Label####
#Read the point metadata by its unique identifier
point = points_list.loc['505_0_3001627']
#Create a label for the point
point_label = re.sub('\.+','_',point['Johnson Controls Name'])
#Add building prefix
point_label = EBU3B[point_label]
#print point_label
#Add to Schema
g.add((point_label, RDF.type, OWL.NamedIndividual))

####Adding Point Type###
#Look up the schema tagset
point_type = desc_tagset_map.loc[point['BACnet Description']]['Schema Label']
#Get the first match, capitalize the first letter of each word and concatenate by '_'
point_type = '_'.join(point_type[0].title().split())
#Concatenate with brick prefix
point_type = BRICK[point_type]
#print point_type
#Check if point type exists in the schema
if (point_type, None, None) in g:
    #Add point type to schema
    g.add((point_label, RDF.type, point_type))
else:
    print "Could not identify point type in schema:", point_type 

In [14]:
###Example for a VAV instance ####
#Continuing with the point information above

##Create Equipment Label##
equip_label = point['Building'] + '_' + point['Equipment Type'] + '_' + point['Location']
#replaces spaces with '_'
equip_label = '_'.join(equip_label.split())
#add building prefix
equip_label = EBU3B[equip_label]
#print equip_label
#add to schema instance
g.add((equip_label, RDF.type, OWL.NamedIndividual))

##Add equipment type##
equip_type = point['Equipment Type']
#add brick prefix
equip_type = BRICK[equip_type]
#print equip_type
#Check if equip type exists in the schema
if (equip_type, None, None) in g:
    #Add equip type to schema
    g.add((equip_label, RDF.type, equip_type))
else:
    print "Could not identify equipment type in schema:", equip_type 
    
##Add relationship between point and equipment ##
g.add((equip_label, BRICK.hasPoint, point_label))

In [16]:
# Store output as .ttl
g.serialize(destination='ebu3b_brick.ttl', format='turtle')
# Search for "EBU3B" in the output file, you'll find the instances we created.

In [17]:
#TODO
#Iterate through all the points in the building and map them to schema
for point_id, point_metadata in points_list.iterrows():
    print point_metadata
    break

BACnet Device ID                                    505
BACnet Object Type                                    0
BACnet Object ID                                3001627
Johnson Controls Name          EBU3B.RM-1100A..ACTCLGSP
BACnet Name                NAE 05 N2 2 VMA 103 ACTCLGSP
BACnet Description              Actual Cooling Setpoint
BACnet Unit                                          64
Ground Truth Point Type         Actual Cooling Setpoint
Point Ref                                           NaN
Location                                       RM-1100A
Equipment Type                                      VAV
Equipment Ref                                       103
Equipment Subsystems                                NaN
Building                                          EBU3B
Comments                                            NaN
BACnet Type Str                            Analog Input
Name: 505_0_3001627, dtype: object


In [18]:
#### For Graph Traversal #####
#For traversing the Brick or EBU3B graph, use the "Resource" part of RDFlib
#Tutorial here: http://rdflib.readthedocs.io/en/stable/apidocs/rdflib.html#module-rdflib.resource
#Example subclass traversal is shown below
valve = rdflib.resource.Resource(g, BRICK.Valve)
subclasses = list(valve.transitive_subjects(RDFS.subClassOf))
[c.qname() for c in subclasses]

[u'Valve', u'Reheat_Valve', u'Heating_Valve', u'Cooling_Valve']